In [16]:
from flask import Flask, render_template, request, send_from_directory, redirect, url_for, flash
import os
import pandas as pd
import lxml
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import cx_Oracle
app = Flask(__name__)
app.secret_key = 'your_secret_key_here'
app.config['MAX_CONTENT_LENGTH'] = 20 * 1024 * 1024 * 1024  



Conncting to oracle Database


In [17]:
##dsn_tns = cx_Oracle.makedsn('pc0308', '1521', service_name='XE')
##conn = cx_Oracle.connect(user='ivan', password='password', dsn=dsn_tns)

Uploading csv file (later maybe removed and replaced with database)

In [18]:
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

uploaded_tables = {}  

@app.route('/', methods=['GET', 'POST'])
def upload_files():
    global uploaded_tables
    
    if request.method == 'POST':
        files = request.files.getlist('file')

        for file in files:
            if file.filename != '':
                file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
                file.save(file_path)
                data = pd.read_csv(file_path)
                uploaded_tables[file.filename] = data.to_html(index=False) 
    
    return render_template('index.html', uploaded_files=uploaded_tables.keys())



@app.route('/uploads/<filename>')
def display_uploaded(filename):
    table_html = uploaded_tables.get(filename, None)
    if table_html is not None:
        
        df = pd.read_html(table_html, header=0)[0]
        
       
        column_options = [col for col in df.columns if not col.startswith("Unnamed")]
        
        return render_template('uploaded.html', filename=filename, table=table_html, column_options=column_options)
    else:
        return "File not found."




In [19]:
@app.route('/all_tables', methods=['GET'])
def display_all_tables():
   
    if uploaded_tables:
        common_columns = set(pd.read_html(next(iter(uploaded_tables.values())), flavor='html5lib')[0].columns)
        for table_html in uploaded_tables.values():
            df = pd.read_html(table_html, flavor='html5lib')[0]
            common_columns.intersection_update(df.columns)
    else:
        common_columns = set()
    
    return render_template('all_tables.html', uploaded_tables=uploaded_tables, common_columns=common_columns)


Merging

In [20]:
@app.route('/merge_tables', methods=['POST'])
def merge_tables():
    tables_to_merge = request.form.getlist('tables_to_merge')
    merge_attribute = request.form.get('merge_attribute')

    dfs = [pd.read_html(uploaded_tables[filename], flavor='html5lib')[0] for filename in tables_to_merge]

   
    merged_df = dfs[0]
    
   
    for df in dfs[1:]:
        merged_df = pd.merge(merged_df, df, on=merge_attribute, how='outer')

    
    merged_filename = "merged_" + "_".join(tables_to_merge)
    uploaded_tables[merged_filename] = merged_df.to_html(index=False)

    return redirect(url_for('display_all_tables'))


Option to remove unwanted columns

In [21]:
@app.route('/uploads/<filename>/remove_column', methods=['POST'])
def remove_column(filename):
    selected_column = request.form.get('selected_column')
    
    table_html = uploaded_tables.get(filename, None)
    if table_html:
        df = pd.read_html(table_html, header=0)[0]
    else:
        flash("Error accessing the DataFrame.", 'error')
        return redirect(url_for('display_uploaded', filename=filename))

    if selected_column:
        try:
            df.drop(columns=[selected_column], inplace=True)
            uploaded_tables[filename] = df.to_html()  
            flash(f"Column '{selected_column}' has been removed from the DataFrame.", 'success')
        except KeyError:
            flash(f"Column '{selected_column}' not found in the DataFrame.", 'error')
    else:
        flash("Please select a column to remove.", 'error')

    return redirect(url_for('display_uploaded', filename=filename))


In [22]:
def rename_column_and_values(df, old_column_name, new_column_name_prefix):
  
    unique_values = df[old_column_name].unique()
    value_mapping = {unique_values[i]: f"{new_column_name_prefix}_{i+1}" for i in range(len(unique_values))}
    df[new_column_name_prefix] = df[old_column_name].map(value_mapping)
    
    if old_column_name != new_column_name_prefix:
        df = df.drop(old_column_name, axis=1)
    
    return df


In [23]:
@app.route('/uploads/<filename>/rename_column', methods=['POST'])
def rename_column(filename):
    old_column_name = request.form.get('old_column_name')
    new_column_name_prefix = request.form.get('new_column_name_prefix')

    table_html = uploaded_tables.get(filename, None)
    if table_html:
        df = pd.read_html(table_html, header=0)[0]
    else:
        flash("Error accessing the DataFrame.", 'error')
        return redirect(url_for('display_uploaded', filename=filename))

    if old_column_name and new_column_name_prefix:
        df = rename_column_and_values(df, old_column_name, new_column_name_prefix)
        uploaded_tables[filename] = df.to_html(index=False)
        flash(f"Column '{old_column_name}' and its values have been renamed.", 'success')
    else:
        flash("Please select a column and enter a new prefix.", 'error')

    return redirect(url_for('display_uploaded', filename=filename))


Fucntions to preform on a dataset

In [24]:
@app.route('/uploads/<filename>', methods=['POST'])
def apply_action(filename):
    table_html = uploaded_tables.get(filename, None)
    if table_html is not None:
        action = request.form.get('action')
        data_html = uploaded_tables[filename]
        data = pd.read_html(data_html, flavor='html5lib')[0]
        if action == 'remove_duplicates':
            data = remove_duplicates(data)
        elif action == 'remove_missing_values':
            data = remove_missing_values(data)
        elif action == 'convert_boolean_to_binary':
            data = convert_boolean_to_binary(data)
        elif action == 'min_max_scaling':
            data = min_max_scaling(data)
        elif action == 'z_score_normalization':
            data = z_score_normalization(data)
        uploaded_tables[filename] = data.to_html(index=False)
        return redirect(url_for('display_uploaded', filename=filename))
    else:
        return "File not found."

def convert_boolean_to_binary(df):
    return df.applymap(lambda x: int(x) if isinstance(x, bool) else x)

def remove_duplicates(df):
    return df.drop_duplicates()

def remove_missing_values(df):
    return df.dropna()

def min_max_scaling(df):
    scaler = MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

def z_score_normalization(df):
    scaler = StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


In [25]:
# add missing data finder
# add missing data filling


In [26]:
def split_datetime_column(df, datetime_col='DateTime'):
    """Splits a combined date-time column into separate date and time columns."""
    df[datetime_col] = pd.to_datetime(df[datetime_col])
    df['Date'] = df[datetime_col].dt.date
    df['Time'] = df[datetime_col].dt.time
    return df

In [27]:
@app.route('/uploads/<filename>/split_datetime', methods=['POST'])
def split_datetime(filename):
    datetime_column = request.form.get('datetime_column')
    
    table_html = uploaded_tables.get(filename, None)
    if table_html:
        df = pd.read_html(table_html, header=0)[0]
        df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

        # Split the combined date-time column
        df = split_datetime_column(df, datetime_col=datetime_column)

        uploaded_tables[filename] = df.to_html(index=False)
        flash(f"Date-Time column '{datetime_column}' has been split.", 'success')
    else:
        flash("Error accessing the DataFrame.", 'error')

    return redirect(url_for('display_uploaded', filename=filename))


Resempling data

In [28]:
def resample_data_on_column(df, datetime_col='DateTime', interval=15):
    """Resamples the data based on the provided interval."""
    df.set_index(datetime_col, inplace=True)
    df_resampled = df.resample(f'{interval}T').mean()
    
    def handle_outliers(row):
        minute = row.name.minute
        if minute % interval < interval / 2:
            return (minute // interval) * interval
        else:
            return ((minute // interval) + 1) * interval

    df_resampled['NewMinute'] = df_resampled.apply(handle_outliers, axis=1)
    
    return df_resampled

In [29]:
@app.route('/uploads/<filename>/resample', methods=['POST'])
def resample_route(filename):
    interval = request.form.get('interval', type=int)
    datetime_col = request.form.get('datetime_column')  # Fetch the selected datetime column
    
    if not interval:
        flash("Please specify a resampling interval.", 'error')
        return redirect(url_for('display_uploaded', filename=filename))

    table_html = uploaded_tables.get(filename, None)
    if table_html:
        df = pd.read_html(table_html, header=0)[0]
        df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
        
        if datetime_col not in df.columns:
            flash(f"The specified date-time column '{datetime_col}' does not exist.", 'error')
            return redirect(url_for('display_uploaded', filename=filename))

        df_resampled = resample_data_on_column(df, datetime_col=datetime_col, interval=interval)

        uploaded_tables[filename] = df_resampled.to_html(index=False)
        flash(f"Data has been resampled to {interval}-minute intervals.", 'success')
    else:
        flash("Error accessing the DataFrame.", 'error')

    return redirect(url_for('display_uploaded', filename=filename))


In [30]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.216.115:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Sep/2023 12:00:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2023 12:00:53] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Sep/2023 12:00:56] "GET /uploads/train.csv HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2023 12:00:56] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Sep/2023 12:01:10] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2023 12:01:10] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Sep/2023 12:01:11] "GET /uploads/Plant_1_Generation_Data%20-%20Copy.csv HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2023 12:01:11] "GET /static/styles.css HTTP/1.1" 304 -
C:\Users\jovev\AppData\Local\Temp\ipykernel_10228\660582031.py:3: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[datetime_col